In [19]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# our API access function. This will be called by the button when it is clicked
def access_api(button):
    # Ignore the button parameter
    #
    # put code for polling the API here
    #
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again.
    apibutton.icon="check"
    apibutton.disabled=True

# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

In [2]:
lineagedf=pd.read_pickle("lineagedf.pkl")

In [8]:
year=wdg.Select(
    # options available: unique years in the dataframe
    options=lineagedf.index.year.unique(), # options available
    value=lineagedf.index.year[-1], # initial value: most recent year
    rows=1, # rows of the selection box
    description='Year',
    disabled=False
)

def lineage_graph(graphyear):
    # our callback function.
    yeardf=lineagedf[lineagedf.index.year==graphyear]
    # average the rows by month
    monthly= yeardf.groupby(pd.Grouper(freq='1M')).mean() 
    totals=monthly.sum(axis=1) # over the rows
    # make sure it's all normalised to 100
    monthly=monthly.div(totals, axis=0)*100
    # older dates on top of the graph
    monthly = monthly[::-1]
    ax=monthly.plot(kind='barh', stacked=True,cmap='tab20')
    ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
    ax.set_yticklabels(monthly.index.strftime('%Y-%m-%d'))
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(lineage_graph, {'graphyear': year})

display(year, output)

Select(description='Year', index=3, options=(2021, 2022, 2023, 2024), rows=1, value=2024)

Output()

In [21]:
timeseriesdf=pd.read_pickle("timeseriesdf.pkl")

print(timeseriesdf)
series=wdg.SelectMultiple(
    options=['cases', 'admissions', 'deaths'],
    value=['cases', 'admissions', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False,
    layout=wdg.Layout(
        width='300px',      
        height='auto',     
        border='none',  
        margin="10px"
    ),
     style={'description_width': '100px'}
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#   value='pineapple', # Defaults to 'pineapple'
#   layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.VBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

            cases admissions deaths
2020-01-30    1.0        0.0    1.0
2020-01-31    0.0        0.0    0.0
2020-02-01    0.0        0.0    0.0
2020-02-02    1.0        0.0    1.0
2020-02-03   18.0        0.0    0.0
...           ...        ...    ...
2024-11-09  149.0        0.0    0.0
2024-11-10  140.0        0.0    0.0
2024-11-11  143.0        0.0    0.0
2024-11-12  110.0        0.0    0.0
2024-11-13   40.0        0.0    0.0

[1750 rows x 3 columns]


Output()